# Setup: Azure AI Foundry Project (Resource Group + AIServices + Project)

----
This notebook bootstraps the **minimum Azure resources** needed for the rest of the lab:

- Azure authentication (CLI/SDK)
- A Resource Group
- An Azure AI Foundry **AIServices** resource and a **Project** subresource
- API key + Project endpoint discovery
- Saving the resulting configuration to a local file for reuse

Run cells from top to bottom. If you hit an auth error, start with the Azure login/authentication section again.

✔️ This setup guide is a revised version of @junwoojeong100's Repository. 

## Table of contents

- [Prerequisites](#prerequisites)
- [Python virtual environment](#python-virtual-environment)
- [Install Python packages](#install-python-packages)
- [Azure authentication (SDK)](#azure-authentication-sdk)
- [Create a Resource Group](#create-a-resource-group)
- [Create Foundry resources (AIServices + Project)](#create-foundry-resources-aiservices--project)
- [Get API key and Project endpoint](#get-api-key-and-project-endpoint)
- [Save configuration for later notebooks](#save-configuration-for-later-notebooks)
- [Next steps](#next-steps)

## Learning goals

- Create an Azure Resource Group using Azure CLI
- Create Azure AI Foundry resources (AIServices + Project) using Azure CLI/ARM REST
- Practice Infrastructure as Code (IaC) concepts with reproducible setup steps

## Prerequisites

### Required

1. **Azure CLI installed**
   - Run the next cell to confirm `az` is available.
   - If you don’t have it installed: https://learn.microsoft.com/cli/azure/install-azure-cli

2. **Azure sign-in**
   - You must be able to authenticate to an Azure tenant and subscription.
   - If you prefer CLI auth, you can run `az login` in a terminal (outside the notebook).

## Python virtual environment

Using a virtual environment keeps your Python dependencies isolated per project.


### Create a virtual environment

```bash
uv sync --prerelease=allow
```

### Activate the virtual environment

```bash
source .venv/bin/activate
```
### VS Code + Jupyter kernel

If you run notebooks in VS Code:
1. Create the venv
2. Click **Select Kernel** (top-right)
3. Choose the **.venv** interpreter

All cells will then run using the virtual environment.

In [ ]:
# Ensure the notebook kernel can find Azure CLI (`az`) on PATH
import os
import subprocess
from dotenv import load_dotenv

load_dotenv(override=True)

possible_paths = [
    "/opt/homebrew/bin",  # macOS (Apple Silicon)
    "/usr/local/bin",     # macOS (Intel) / Linux
    "/usr/bin",           # Linux / Codespaces
    "/home/linuxbrew/.linuxbrew/bin",  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(["which", "az"], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
        print(f"🔍 Azure CLI found: {result.stdout.strip()}")
except Exception:
    pass

paths_to_add: list[str] = []
if az_path and az_path not in os.environ.get("PATH", ""):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get("PATH", ""):
            paths_to_add.append(path)

if paths_to_add:
    os.environ["PATH"] = ":".join(paths_to_add) + ":" + os.environ.get("PATH", "")
    print(f"✅ Added to PATH: {', '.join(paths_to_add)}")
else:
    print("✅ PATH looks good already")

print(f"\nPATH (first 150 chars): {os.environ['PATH'][:150]}...")

In [ ]:
# Azure authentication (Python SDK)
from azure.identity import DeviceCodeCredential, InteractiveBrowserCredential
from azure.mgmt.resource import SubscriptionClient
import os

# Optional. If set, the credential will be constrained to this tenant.
TENANT_ID = os.getenv("AZURE_TENANT_ID")

# Detect Codespaces / remote environments where opening a browser may be hard.
is_codespaces = os.getenv("CODESPACES") == "true" or os.getenv("CODESPACE_NAME") is not None
is_remote = os.getenv("REMOTE_CONTAINERS") == "true" or is_codespaces

print("🔐 Starting Azure authentication...")
print(f"Tenant ID: {TENANT_ID or '(not set)'}")

try:
    if is_remote:
        print("\n💡 Remote environment detected.")
        print("Using device-code authentication.\n")
        credential = DeviceCodeCredential(tenant_id=TENANT_ID) if TENANT_ID else DeviceCodeCredential()
        print("📱 Follow the prompt to authenticate:")
        print("   1. Open the URL in your local browser")
        print("   2. Enter the code shown")
        print("   3. Sign in with your Azure account\n")
    else:
        print("A browser window will open for sign-in (if possible).\n")
        credential = (
            InteractiveBrowserCredential(tenant_id=TENANT_ID)
            if TENANT_ID
            else InteractiveBrowserCredential()
        )

    subscription_client = SubscriptionClient(credential)
    subscriptions = list(subscription_client.subscriptions.list())

    print("✅ Authentication succeeded")
    print("\n" + "=" * 80)
    print("📋 Available subscriptions")
    print("=" * 80)

    for i, sub in enumerate(subscriptions, 1):
        print(f"\n{i}. {sub.display_name}")
        print(f"   Subscription ID: {sub.subscription_id}")
        print(f"   State: {sub.state}")

    print("\n" + "=" * 80)
    print(f"✅ Found {len(subscriptions)} subscription(s)")

    # Store a default subscription ID for later cells.
    # If you have multiple subscriptions, you can override AZURE_SUBSCRIPTION_ID manually.
    if subscriptions:
        default_sub = subscriptions[0]
        os.environ["AZURE_SUBSCRIPTION_ID"] = default_sub.subscription_id
        print(f"\n💡 Default subscription set to: {default_sub.display_name}")
        print(f"   AZURE_SUBSCRIPTION_ID={default_sub.subscription_id}")

except Exception as e:
    print(f"\n⚠️ Authentication failed: {e}")
    print("\nTroubleshooting:")
    print("1. Ensure you have access to the tenant/subscription")
    print("2. If using a tenant restriction, verify AZURE_TENANT_ID")
    if is_remote:
        print("3. Ensure you completed the device-code flow in a browser")
    else:
        print("3. Ensure the browser sign-in completed successfully")

## Create a Resource Group

A Resource Group is a logical container for Azure resources.

In [ ]:
RESOURCE_GROUP = "msfoundry-rg"
LOCATION = "northcentralus"

# Create a Resource Group (Azure CLI)
# Note: This uses a fixed name/location used throughout the notebook.
!az group create \
    --name $RESOURCE_GROUP \
    --location $LOCATION # hosted agent only available in northcentralus

# Verify
!az group show \
    --name $RESOURCE_GROUP \
    --output table

## Create Foundry resources (AIServices + Project)

This section creates:

- An **Azure AI Foundry** resource (ARM type: `Microsoft.CognitiveServices/accounts`, `kind=AIServices`)
- A **Project** as a subresource under that Foundry resource

The notebook uses `az rest` with `api-version=2025-04-01-preview` because it supports project management settings.

In [ ]:
# Configuration used by the provisioning steps
import os
import random
import string

# Generate a unique resource name suffix (to avoid collisions)
random_suffix = "".join(random.choices(string.ascii_lowercase + string.digits, k=6))
FOUNDRY_NAME = f"foundry-{random_suffix}"
PROJECT_NAME = "msfoundry-prj1"


print("✅ Configuration set")
print(f"   Foundry resource name: {FOUNDRY_NAME}")
print(f"   Project name:         {PROJECT_NAME}")
print(f"   Resource group:       {RESOURCE_GROUP}")
print(f"   Location:             {LOCATION}")

In [ ]:
# Step 1: Create the Foundry resource (AIServices)
import json
import subprocess

subscription_id = os.environ.get("AZURE_SUBSCRIPTION_ID", "")
if not subscription_id:
    raise RuntimeError("AZURE_SUBSCRIPTION_ID is not set. Run the Azure authentication cell first.")

# Uses 2025-04-01-preview to support allowProjectManagement
foundry_url = (
    f"https://management.azure.com/subscriptions/{subscription_id}"
    f"/resourceGroups/{RESOURCE_GROUP}"
    f"/providers/Microsoft.CognitiveServices/accounts/{FOUNDRY_NAME}"
    "?api-version=2025-04-01-preview"
)

foundry_body = json.dumps(
    {
        "location": LOCATION,
        "kind": "AIServices",
        "sku": {"name": "S0"},
        "identity": {"type": "SystemAssigned"},
        "tags": {
            "SecurityControl": "Ignore",
        },
        "properties": {
            "customSubDomainName": FOUNDRY_NAME,
            "publicNetworkAccess": "Enabled",
            "allowProjectManagement": True,
            "disableLocalAuth": False,
        },
    }
)

print(f"📌 Step 1: Creating Foundry resource: {FOUNDRY_NAME}")
print("   Type: AIServices")
print("   API version: 2025-04-01-preview")
print("   allowProjectManagement: True")
print("   disableLocalAuth: False")
print("   tags: SecurityControl=Ignore")

result = subprocess.run(
    ["az", "rest", "--method", "PUT", "--url", foundry_url, "--body", foundry_body],
    capture_output=True,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Foundry resource creation failed: {result.stderr}")

print("✅ Foundry resource created")
foundry_info = json.loads(result.stdout)
foundry_id = foundry_info.get("id", "")
os.environ["FOUNDRY_ID"] = foundry_id

properties = foundry_info.get("properties", {})
allow_project = properties.get("allowProjectManagement")
disable_local_auth = properties.get("disableLocalAuth")

if not disable_local_auth:
    print("   ✅ enabled API Key authentication")
else:
    print("   ⚠️ disabled API Key authentication")

print(f"   Foundry ID: {foundry_id}...")
print(f"   allowProjectManagement: {allow_project}")
print(f"   disableLocalAuth: {disable_local_auth}")

In [ ]:
# Step 2: Create a Foundry Project (subresource)
import json
import subprocess
import time

print(f"📌 Step 2: Creating Foundry project: {PROJECT_NAME}")

foundry_id = os.environ.get("FOUNDRY_ID", "")
if not foundry_id:
    raise RuntimeError("FOUNDRY_ID is not set. Run Step 1 first.")

# Wait a bit for ARM to finish provisioning the parent resource
time.sleep(5)

project_url = (
    f"https://management.azure.com{foundry_id}/projects/{PROJECT_NAME}"
    "?api-version=2025-04-01-preview"
)

project_body = json.dumps(
    {
        "location": LOCATION,
        "identity": {"type": "SystemAssigned"},
        "tags": {
            "SecurityControl": "Ignore",
        },
        "properties": {
            "friendlyName": PROJECT_NAME,
            "description": f"Foundry Project: {PROJECT_NAME}",
        },
    }
)

result = subprocess.run(
    ["az", "rest", "--method", "PUT", "--url", project_url, "--body", project_body],
    capture_output=True,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Project creation failed: {result.stderr}")

print("✅ Foundry project created")
project_info = json.loads(result.stdout)
project_id = project_info.get("id", "")
os.environ["PROJECT_ID"] = project_id
print(f"   Project ID: {project_id[:70]}...")

In [ ]:
# Verify the created resources
import subprocess

print("📋 Verifying resources...\n")

result = subprocess.run(
    [
        "az",
        "cognitiveservices",
        "account",
        "show",
        "--name",
        FOUNDRY_NAME,
        "--resource-group",
        RESOURCE_GROUP,
        "--query",
        "{Name:name, Kind:kind, Location:location, Endpoint:properties.endpoint}",
        "--output",
        "table",
    ],
    capture_output=True,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Verification failed: {result.stderr}")

print("Foundry resource:")
print(result.stdout)
print("✅ Foundry resource and project were created")
print("💡 Foundry portal: https://ai.azure.com")

## Get API key and Project endpoint

This section retrieves the **AIServices API key** and constructs a **Project endpoint** URL that other notebooks can reuse.

In [ ]:
# Get the AIServices API key and build the Project endpoint
import json
import subprocess

print("📌 Retrieving Foundry (AIServices) API key")
print("💡 Foundry Projects typically use the parent AIServices keys.\n")

foundry_id = os.environ.get("FOUNDRY_ID", "")
# foundry_id = "/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/AOAI-group3/providers/Microsoft.CognitiveServices/accounts/hyo-msfoundry-pjt1-resource/projects/hyo-msfoundry-pjt1"
if not foundry_id:
    raise RuntimeError("FOUNDRY_ID is not set. Run the provisioning steps first.")

# Method 1: Azure CLI
print("🔑 Method 1: Azure CLI (cognitiveservices account keys list)")
result = subprocess.run(
    [
        "az",
        "cognitiveservices",
        "account",
        "keys",
        "list",
        "--name",
        FOUNDRY_NAME,
        "--resource-group",
        RESOURCE_GROUP,
    ],
    capture_output=True,
    text=True,
)

primary_key = ""
if result.returncode == 0:
    keys = json.loads(result.stdout)
    primary_key = keys.get("key1", "")

if not primary_key:
    print("⚠️ CLI key retrieval failed or returned empty key.")
    print("🔑 Method 2: ARM REST (listKeys)")
    key_url = f"https://management.azure.com{foundry_id}/listKeys?api-version=2025-04-01-preview"
    result2 = subprocess.run(
        ["az", "rest", "--method", "POST", "--url", key_url],
        capture_output=True,
        text=True,
)
    if result2.returncode != 0:
        raise RuntimeError(f"Key retrieval failed: {result2.stderr}")
    keys = json.loads(result2.stdout)
    primary_key = keys.get("key1", "") or keys.get("primaryKey", "")

if not primary_key:
    raise RuntimeError("Could not retrieve a non-empty API key.")

os.environ["FOUNDRY_API_KEY"] = primary_key
print("✅ API key retrieved")

# Retrieve the base endpoint from the AIServices resource
endpoint_result = subprocess.run(
    [
        "az",
        "cognitiveservices",
        "account",
        "show",
        "--name",
        FOUNDRY_NAME,
        "--resource-group",
        RESOURCE_GROUP,
        "--query",
        "properties.endpoint",
        "--output",
        "tsv",
    ],
    capture_output=True,
    text=True,
)
if endpoint_result.returncode != 0:
    raise RuntimeError(f"Endpoint retrieval failed: {endpoint_result.stderr}")

base_endpoint = endpoint_result.stdout.strip()
if not base_endpoint:
    raise RuntimeError("Base endpoint is empty.")

# Construct a project endpoint (this matches the existing notebook convention)
project_endpoint = (
    base_endpoint.replace(
        ".cognitiveservices.azure.com/",
        ".services.ai.azure.com/",
    ).rstrip("/")
    + f"/api/projects/{PROJECT_NAME}"
)

os.environ["AZURE_AI_PROJECT_ENDPOINT"] = project_endpoint
print(f"   Base endpoint:    {base_endpoint}")
print(f"   Azure AI Project endpoint: {project_endpoint}")

print("\n💡 You can also verify Keys & Endpoint in Azure Portal or Foundry portal:")
print("   - https://portal.azure.com")
print("   - https://ai.azure.com")

## Save configuration for later notebooks

This writes the key outputs to a local JSON file so other notebooks can load them without re-provisioning.

In [ ]:
# Save configuration to a local JSON file
import json
import os

config = {
    "FOUNDRY_NAME": FOUNDRY_NAME,
    "PROJECT_NAME": PROJECT_NAME,
    "RESOURCE_GROUP": RESOURCE_GROUP,
    "LOCATION": LOCATION,
    "AZURE_SUBSCRIPTION_ID": os.environ.get("AZURE_SUBSCRIPTION_ID", ""),
    "TENANT_ID": os.getenv("AZURE_TENANT_ID", ""),
    "FOUNDRY_ID": os.environ.get("FOUNDRY_ID", ""),
    "PROJECT_ID": os.environ.get("PROJECT_ID", ""),
    "AZURE_AI_PROJECT_ENDPOINT": os.environ.get("AZURE_AI_PROJECT_ENDPOINT", ""),
}

config_file = ".foundry_config.json"
with open(config_file, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2)

print(f"✅ Saved config file: {config_file}")
print("\n📋 Summary:")
print(f"   Foundry:   {FOUNDRY_NAME}")
print(f"   Project:   {PROJECT_NAME}")
print(f"   Location:  {LOCATION}")
print(f"   Endpoint:  {os.environ.get('AZURE_AI_PROJECT_ENDPOINT', 'N/A')}")
print("   API key:   not saved (use AAD auth or env)")
print("\n💡 Next notebooks can load this file to reuse settings.")

## Optional: verify in Azure Portal

To visually confirm resources:

1. Go to https://portal.azure.com
2. Open Resource Group **foundry-code**
3. Select the AIServices resource **{FOUNDRY_NAME}** (name shown in the output above) to view **Keys and Endpoint**

You can also confirm the Project in the Foundry portal: https://ai.azure.com

## Optional: Revise the Foundry resource with different settings

This section revises the existing Foundry resource with different disableLocalAuth.

In [ ]:
import json
import subprocess

subscription_id = "<your subscription id>"
resource_group = "<your resource group>"
foundry_parent_resource_name = "<your foundry name>"
location = "northcentralus"

# Uses 2025-04-01-preview to support allowProjectManagement
foundry_url = (
    f"https://management.azure.com/subscriptions/{subscription_id}"
    f"/resourceGroups/{resource_group}"
    f"/providers/Microsoft.CognitiveServices/accounts/{foundry_parent_resource_name}"
    "?api-version=2025-04-01-preview"
)

foundry_body = json.dumps(
    {
        "kind": "AIServices",
        "sku": {"name": "S0"},
        "location": f"{location}",
        "identity": {"type": "SystemAssigned"},
        "tags": {
            "SecurityControl": "Ignore",
        },
        "properties": {
            "networkAcls": {
                "defaultAction": "Allow",
            },
            "disableLocalAuth": False,
        },
    }
)

print(f"📌 Step 1: Modified Foundry resource: {foundry_parent_resource_name}")
print("   Type: AIServices")
print("   disableLocalAuth: False")
print("   tags: SecurityControl=Ignore")

result = subprocess.run(
    ["az", "rest", "--method", "PATCH", "--url", foundry_url, "--body", foundry_body],
    capture_output=True,
    text=True,
)

if result.returncode != 0:
    raise RuntimeError(f"Foundry resource creation failed: {result.stderr}")

print("✅ Foundry resource created")
foundry_info = json.loads(result.stdout)
foundry_id = foundry_info.get("id", "")
print(f"   Foundry ID: {foundry_id}")

properties = foundry_info.get("properties", {})
allow_project = properties.get("allowProjectManagement")
disable_local_auth = properties.get("disableLocalAuth")

print(f"    disableLocalAuth: {disable_local_auth}")

📌 Step 1: Modified Foundry resource: hyo-ai-foundry-pjt2-wes-resource
   Type: AIServices
   disableLocalAuth: False
   tags: SecurityControl=Ignore
✅ Foundry resource created
   Foundry ID: /subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/AOAI-rg-westus/providers/Microsoft.CognitiveServices/accounts/hyo-ai-foundry-pjt2-wes-resource
    disableLocalAuth: False


## Additional resources

- Azure AI Foundry overview: https://learn.microsoft.com/azure/ai-foundry/what-is-azure-ai-foundry?view=foundry
- Azure Resource Manager overview: https://learn.microsoft.com/azure/azure-resource-manager/management/overview
- Azure regions and availability zones: https://learn.microsoft.com/azure/reliability/availability-zones-overview